## Chatbot Interaktif Phi 3 Mini 4K Instruct dengan Whisper

### Pengenalan:
Chatbot Interaktif Phi 3 Mini 4K Instruct adalah alat yang membolehkan pengguna berinteraksi dengan demo Microsoft Phi 3 Mini 4K Instruct menggunakan input teks atau audio. Chatbot ini boleh digunakan untuk pelbagai tugas, seperti terjemahan, kemas kini cuaca, dan pengumpulan maklumat umum.


In [ ]:
#Install required Python Packages
!pip install accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install flash-attn --no-build-isolation', env={'FLASH_ATTENTION_SKIP_CUDA_BUILD': "TRUE"}, shell=True
!pip install transformers
!pip install wheel
!pip install gradio
!pip install pydub==0.25.1
!pip install edge-tts
!pip install openai-whisper==20231117
!pip install ffmpeg==1.4
# from IPython.display import clear_output
# clear_output()

In [ ]:
# Checking to see if Cuda support is available 
# Output True = Cuda
# Output False = No Cuda (installing Cuda will be required to run the model on GPU)
import os 
import torch
print(torch.cuda.is_available())


Buat Token Akses Huggingface Anda

Buat token baharu  
Berikan nama baharu  
Pilih kebenaran menulis  
Salin token dan simpan di tempat yang selamat


Kod Python berikut menjalankan dua tugas utama: mengimport modul `os` dan menetapkan pemboleh ubah persekitaran.

1. Mengimport modul `os`:
   - Modul `os` dalam Python menyediakan cara untuk berinteraksi dengan sistem operasi. Ia membolehkan anda melakukan pelbagai tugas berkaitan sistem operasi, seperti mengakses pemboleh ubah persekitaran, bekerja dengan fail dan direktori, dan sebagainya.
   - Dalam kod ini, modul `os` diimport menggunakan pernyataan `import`. Pernyataan ini menjadikan fungsi modul `os` tersedia untuk digunakan dalam skrip Python semasa.

2. Menetapkan pemboleh ubah persekitaran:
   - Pemboleh ubah persekitaran ialah nilai yang boleh diakses oleh program yang berjalan pada sistem operasi. Ia adalah cara untuk menyimpan tetapan konfigurasi atau maklumat lain yang boleh digunakan oleh pelbagai program.
   - Dalam kod ini, pemboleh ubah persekitaran baru ditetapkan menggunakan kamus `os.environ`. Kunci kamus ialah `'HF_TOKEN'`, dan nilainya diberikan daripada pemboleh ubah `HUGGINGFACE_TOKEN`.
   - Pemboleh ubah `HUGGINGFACE_TOKEN` ditakrifkan tepat di atas petikan kod ini, dan ia diberikan nilai rentetan `"hf_**************"` menggunakan sintaks `#@param`. Sintaks ini sering digunakan dalam Jupyter notebook untuk membolehkan input pengguna dan konfigurasi parameter secara langsung dalam antara muka notebook.
   - Dengan menetapkan pemboleh ubah persekitaran `'HF_TOKEN'`, ia boleh diakses oleh bahagian lain program atau program lain yang berjalan pada sistem operasi yang sama.

Secara keseluruhan, kod ini mengimport modul `os` dan menetapkan pemboleh ubah persekitaran bernama `'HF_TOKEN'` dengan nilai yang diberikan dalam pemboleh ubah `HUGGINGFACE_TOKEN`.


In [ ]:
import os
# set the Hugging Face Token from 
# add the Hugging Face Token to the environment variables
HUGGINGFACE_TOKEN = "Enter Hugging Face Key" #@param {type:"string"}
os.environ['HF_TOKEN']HUGGINGFACE_TOKEN

Petikan kod ini mentakrifkan fungsi yang dipanggil clear_output yang digunakan untuk membersihkan output sel semasa dalam Jupyter Notebook atau IPython. Mari kita pecahkan kod ini dan fahami fungsinya:

Fungsi clear_output mengambil satu parameter yang dipanggil wait, yang merupakan nilai boolean. Secara lalai, wait ditetapkan kepada False. Parameter ini menentukan sama ada fungsi tersebut perlu menunggu sehingga output baru tersedia untuk menggantikan output sedia ada sebelum membersihkannya.

Fungsi ini sendiri digunakan untuk membersihkan output sel semasa. Dalam Jupyter Notebook atau IPython, apabila satu sel menghasilkan output, seperti teks yang dicetak atau plot grafik, output tersebut dipaparkan di bawah sel. Fungsi clear_output membolehkan anda membersihkan output tersebut.

Pelaksanaan fungsi ini tidak disediakan dalam petikan kod, seperti yang ditunjukkan oleh ellipsis (...). Ellipsis mewakili tempat letak untuk kod sebenar yang melaksanakan pembersihan output. Pelaksanaan fungsi ini mungkin melibatkan interaksi dengan API Jupyter Notebook atau IPython untuk menghapuskan output sedia ada dari sel.

Secara keseluruhan, fungsi ini menyediakan cara yang mudah untuk membersihkan output sel semasa dalam Jupyter Notebook atau IPython, menjadikannya lebih mudah untuk mengurus dan mengemas kini output yang dipaparkan semasa sesi pengekodan interaktif.


In [ ]:
# Download Phi-3-mini-4k-instruct model & Whisper Tiny
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

#whisper for speech to text()
import whisper
select_model ="tiny" # ['tiny', 'base']
whisper_model = whisper.load_model(select_model)

#from IPython.display import clear_output
#clear_output()

Mari kita melalui pelaksanaan fungsi yang relevan satu persatu untuk melakukan teks-ke-ucapan (TTS) menggunakan perkhidmatan Edge TTS:

1. `calculate_rate_string(input_value)`: Fungsi ini mengambil nilai input dan mengira rentetan kadar untuk suara TTS. Nilai input mewakili kelajuan ucapan yang diinginkan, di mana nilai 1 mewakili kelajuan normal. Fungsi ini mengira rentetan kadar dengan menolak 1 daripada nilai input, menggandakannya dengan 100, dan kemudian menentukan tanda berdasarkan sama ada nilai input lebih besar atau sama dengan 1. Fungsi ini mengembalikan rentetan kadar dalam format "{sign}{rate}".

2. `make_chunks(input_text, language)`: Fungsi ini mengambil teks input dan bahasa sebagai parameter. Ia memecahkan teks input kepada bahagian berdasarkan peraturan khusus bahasa. Dalam pelaksanaan ini, jika bahasa adalah "English", fungsi ini memecahkan teks pada setiap titik (".") dan membuang sebarang ruang kosong di awal atau akhir. Ia kemudian menambahkan titik pada setiap bahagian dan mengembalikan senarai bahagian yang telah ditapis.

3. `tts_file_name(text)`: Fungsi ini menjana nama fail untuk fail audio TTS berdasarkan teks input. Ia melakukan beberapa transformasi pada teks: membuang titik di akhir (jika ada), menukar teks kepada huruf kecil, membuang ruang kosong di awal dan akhir, dan menggantikan ruang dengan garis bawah. Ia kemudian memotong teks kepada maksimum 25 aksara (jika lebih panjang) atau menggunakan teks penuh jika ia kosong. Akhirnya, ia menjana rentetan rawak menggunakan modul [`uuid`] dan menggabungkannya dengan teks yang telah dipotong untuk mencipta nama fail dalam format "/content/edge_tts_voice/{truncated_text}_{random_string}.mp3".

4. `merge_audio_files(audio_paths, output_path)`: Fungsi ini menggabungkan beberapa fail audio menjadi satu fail audio. Ia mengambil senarai laluan fail audio dan laluan output sebagai parameter. Fungsi ini memulakan objek kosong `AudioSegment` yang dipanggil [`merged_audio`]. Ia kemudian mengulangi setiap laluan fail audio, memuatkan fail audio menggunakan kaedah `AudioSegment.from_file()` daripada perpustakaan `pydub`, dan menambahkan fail audio semasa kepada objek [`merged_audio`]. Akhirnya, ia mengeksport audio yang telah digabungkan ke laluan output yang ditentukan dalam format MP3.

5. `edge_free_tts(chunks_list, speed, voice_name, save_path)`: Fungsi ini melakukan operasi TTS menggunakan perkhidmatan Edge TTS. Ia mengambil senarai bahagian teks, kelajuan ucapan, nama suara, dan laluan simpan sebagai parameter. Jika bilangan bahagian lebih daripada 1, fungsi ini mencipta direktori untuk menyimpan fail audio bahagian individu. Ia kemudian mengulangi setiap bahagian, membina arahan Edge TTS menggunakan fungsi `calculate_rate_string()`, nama suara, dan teks bahagian, dan melaksanakan arahan menggunakan fungsi `os.system()`. Jika pelaksanaan arahan berjaya, ia menambahkan laluan fail audio yang dijana ke senarai. Selepas memproses semua bahagian, ia menggabungkan fail audio individu menggunakan fungsi `merge_audio_files()` dan menyimpan audio yang telah digabungkan ke laluan simpan yang ditentukan. Jika hanya ada satu bahagian, ia terus menjana arahan Edge TTS dan menyimpan audio ke laluan simpan. Akhirnya, ia mengembalikan laluan simpan fail audio yang dijana.

6. `random_audio_name_generate()`: Fungsi ini menjana nama fail audio rawak menggunakan modul [`uuid`]. Ia menjana UUID rawak, menukarnya kepada rentetan, mengambil 8 aksara pertama, menambahkan sambungan ".mp3", dan mengembalikan nama fail audio rawak.

7. `talk(input_text)`: Fungsi ini adalah titik masuk utama untuk melakukan operasi TTS. Ia mengambil teks input sebagai parameter. Ia mula-mula memeriksa panjang teks input untuk menentukan sama ada ia adalah ayat panjang (lebih besar atau sama dengan 600 aksara). Berdasarkan panjang dan nilai pemboleh ubah `translate_text_flag`, ia menentukan bahasa dan menjana senarai bahagian teks menggunakan fungsi `make_chunks()`. Ia kemudian menjana laluan simpan untuk fail audio menggunakan fungsi `random_audio_name_generate()`. Akhirnya, ia memanggil fungsi `edge_free_tts()` untuk melakukan operasi TTS dan mengembalikan laluan simpan fail audio yang dijana.

Secara keseluruhan, fungsi-fungsi ini bekerjasama untuk memecahkan teks input kepada bahagian, menjana nama fail untuk fail audio, melakukan operasi TTS menggunakan perkhidmatan Edge TTS, dan menggabungkan fail audio individu menjadi satu fail audio.


In [ ]:
#@title Edge TTS
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      temp_list = input_text.strip().split(".")
      filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      if temp_list[-1].strip():
          filtered_list.append(temp_list[-1].strip())
      return filtered_list


import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    # print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path

# text = "This is Microsoft Phi 3 mini 4k instruct Demo" Simply update the text variable with the text you want to convert to speech
text = 'This is Microsoft Phi 3 mini 4k instruct Demo'  # @param {type: "string"}
Language = "English" # @param ['English']
# Gender of voice simply change from male to female and choose the voice you want to use
Gender = "Female"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
if len(text)>=600:
  long_sentence = True
else:
  long_sentence = False

# long_sentence = False # @param {type:"boolean"}
save_path = ''  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global translate_text_flag,Language,speed,voice_name
  if len(input_text)>=600:
    long_sentence = True
  else:
    long_sentence = False

  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path


edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

Pelaksanaan dua fungsi: convert_to_text dan run_text_prompt, serta deklarasi dua kelas: str dan Audio.

Fungsi convert_to_text mengambil audio_path sebagai input dan menyalin audio kepada teks menggunakan model yang dipanggil whisper_model. Fungsi ini mula-mula memeriksa sama ada bendera gpu disetkan kepada True. Jika ya, whisper_model digunakan dengan parameter tertentu seperti word_timestamps=True, fp16=True, language='English', dan task='translate'. Jika bendera gpu adalah False, whisper_model digunakan dengan fp16=False. Transkripsi yang dihasilkan kemudian disimpan ke dalam fail bernama 'scan.txt' dan dikembalikan sebagai teks.

Fungsi run_text_prompt mengambil mesej dan chat_history sebagai input. Ia menggunakan fungsi phi_demo untuk menghasilkan respons daripada chatbot berdasarkan mesej input. Respons yang dihasilkan kemudian dihantar kepada fungsi talk, yang menukar respons tersebut kepada fail audio dan mengembalikan laluan fail. Kelas Audio digunakan untuk memaparkan dan memainkan fail audio. Audio dipaparkan menggunakan fungsi display dari modul IPython.display, dan objek Audio dicipta dengan parameter autoplay=True, supaya audio mula dimainkan secara automatik. Chat_history dikemas kini dengan mesej input dan respons yang dihasilkan, dan string kosong serta chat_history yang dikemas kini dikembalikan.

Kelas str adalah kelas terbina dalam Python yang mewakili urutan aksara. Ia menyediakan pelbagai kaedah untuk memanipulasi dan bekerja dengan string, seperti capitalize, casefold, center, count, encode, endswith, expandtabs, find, format, index, isalnum, isalpha, isascii, isdecimal, isdigit, isidentifier, islower, isnumeric, isprintable, isspace, istitle, isupper, join, ljust, lower, lstrip, partition, replace, removeprefix, removesuffix, rfind, rindex, rjust, rpartition, rsplit, rstrip, split, splitlines, startswith, strip, swapcase, title, translate, upper, zfill, dan banyak lagi. Kaedah-kaedah ini membolehkan anda melakukan operasi seperti mencari, mengganti, memformat, dan memanipulasi string.

Kelas Audio adalah kelas tersuai yang mewakili objek audio. Ia digunakan untuk mencipta pemain audio dalam persekitaran Jupyter Notebook. Kelas ini menerima pelbagai parameter seperti data, filename, url, embed, rate, autoplay, dan normalize. Parameter data boleh berupa array numpy, senarai sampel, string yang mewakili nama fail atau URL, atau data PCM mentah. Parameter filename digunakan untuk menentukan fail tempatan untuk memuatkan data audio, dan parameter url digunakan untuk menentukan URL untuk memuat turun data audio. Parameter embed menentukan sama ada data audio harus disematkan menggunakan URI data atau dirujuk dari sumber asal. Parameter rate menentukan kadar pensampelan data audio. Parameter autoplay menentukan sama ada audio harus mula dimainkan secara automatik. Parameter normalize menentukan sama ada data audio harus dinormalisasi (diskalakan semula) kepada julat maksimum yang mungkin. Kelas Audio juga menyediakan kaedah seperti reload untuk memuat semula data audio dari fail atau URL, dan atribut seperti src_attr, autoplay_attr, dan element_id_attr untuk mendapatkan atribut yang sepadan untuk elemen audio dalam HTML.

Secara keseluruhan, fungsi dan kelas ini digunakan untuk menyalin audio kepada teks, menghasilkan respons audio daripada chatbot, dan memaparkan serta memainkan audio dalam persekitaran Jupyter Notebook.


In [ ]:
#@title Run gradio app
def convert_to_text(audio_path):
  gpu=True
  if gpu:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=True,language='English',task='translate')
  else:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=False,language='English',task='translate')
  with open('scan.txt', 'w') as file:
    file.write(str(result))
  return result["text"]


import gradio as gr
from IPython.display import Audio, display
def run_text_prompt(message, chat_history):
    bot_message = phi_demo(message)
    edge_save_path=talk(bot_message)
    # print(edge_save_path)
    display(Audio(edge_save_path, autoplay=True))

    chat_history.append((message, bot_message))
    return "", chat_history


def run_audio_prompt(audio, chat_history):
    if audio is None:
        return None, chat_history
    print(audio)
    message_transcription = convert_to_text(audio)
    _, chat_history = run_text_prompt(message_transcription, chat_history)
    return None, chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with Phi 3 mini 4k instruct")

    msg = gr.Textbox(label="Ask anything")
    msg.submit(run_text_prompt, [msg, chatbot], [msg, chatbot])

    with gr.Row():
        audio = gr.Audio(sources="microphone", type="filepath")

        send_audio_button = gr.Button("Send Audio", interactive=True)
        send_audio_button.click(run_audio_prompt, [audio, chatbot], [audio, chatbot])

demo.launch(share=True,debug=True)


---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk memastikan ketepatan, sila ambil perhatian bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asalnya harus dianggap sebagai sumber yang berwibawa. Untuk maklumat yang kritikal, terjemahan manusia profesional adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
